In [1]:
!pip --trusted-host pypi.org --trusted-host files.pythonhosted.org install -r requirements.txt -qq

import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from transformers import *
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from ast import literal_eval

In [2]:
# 환경설정
## 환경변수
VER = 26
# LOAD_TOKENS_FROM = 'input/py-bigbird-v26'
# LOAD_MODEL_FROM = 'input/py-bigbird-v26'
# DOWNLOADED_MODEL_PATH = 'input/py-bigbird-v26'
DOWNLOADED_MODEL_PATH = 'model'
MODEL_NAME = 'google/bigbird-roberta-base'
CONFIG = {
    'model_name': MODEL_NAME,
    'max_length': 1024,
    'train_batch_size': 4,
    'valid_batch_size': 4,
    'epochs': 5,
    'learning_rates': [2.5e-5, 2.5e-5, 2.5e-6, 2.5e-6, 2.5e-7],
    'max_grad_norm': 10,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
}
COMPUTE_VAL_SCORE = True if len(os.listdir('data/test')) <= 5 else False

## 기타설정
os.makedirs('result', exist_ok=True)
os.makedirs('model', exist_ok=True)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
## 모델설정
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, add_prefix_space=True)
tokenizer.save_pretrained('model')

config_model = AutoConfig.from_pretrained(MODEL_NAME)
config_model.num_labels = 15
config_model.save_pretrained('model')

backbone = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, config=config_model)
backbone.save_pretrained('model')

SSLError: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/google/bigbird-roberta-base (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)')))

In [ ]:
# 데이터 로딩
## 다운로드 (using Kaggle API)
# !kaggle competitions download -q -c feedback-prize-2021
# !unzip feedback-prize-2021.zip -d data

## train.csv
train_df = pd.read_csv('data/train.csv')

## train
train_ids, train_texts = [], []
for f in tqdm(os.listdir('data/train')):
    train_ids.append(f.replace('.txt', ''))
    train_texts.append(open(f'data/train/{f}', 'r', encoding='utf8').read())
train_text_df = pd.DataFrame({'id': train_ids, 'text': train_texts})

## test
test_ids, test_texts = [], []
for f in tqdm(os.listdir('data/test')):
    test_ids.append(f.replace('.txt', ''))
    test_texts.append(open(f'data/test/{f}', 'r', encoding='utf8').read())
test_text_df = pd.DataFrame({'id': test_ids, 'text': test_texts})

100%|██████████| 5/5 [00:00<00:00, 5009.92it/s]


In [118]:
all_entities = []
for i, row in train_text_df.iterrows():
    if i%100 == 0: print(i, ', ', end='')
    total = len(row['text'].split())
    id = row['id']
    entities = ['0']*total
    for _, discource in train_df.query("id == @id").iterrows():
        disc_type = discource['discourse_type']
        list_ix = [int(x) for x in discource['predictionstring'].split(' ')]
        entities[list_ix[0]] = f'B-{disc_type}'
        for j in list_ix[1:]:
            entities[j] = f'I-{disc_type}'
        all_entities.append(entities)

449it [00:03, 137.88it/s]